**PROYECTO NLP DPIR 2023**

In [26]:
import pandas as pd
import numpy as np
import spacy
from spacy.tokens import Doc, Span
from spacy.matcher import PhraseMatcher

In [44]:
archivo = pd.read_excel("BD version 1 NLP.xlsx", sheet_name= "Hoja2")
archivo.head(5)


,Renglon,Nombre del Producto,Caracteristicas Solicitadas,Presentacion,Unidad de Medida,Precio de Referencia,Rubro,Nog,Evento,Categoria
0,1,CERA PARA HUESO (PALMITATO ISOPROPÍLICO),CERA PARA HUESO: A BASE DE CERA DE ABEJA (30 ...,NaN,GRAMO,21.500000,MATERIAL HEMOSTATICO,18754341,CONTRATO ABIERTO NÚMERO 02-2022 ADQUISICIÓN DE...,Material medico
1,2,CERA PARA HUESO (JALEA DE PETROLATO),CERA PARA HUESO: A BASE DE CERA DE ABEJA (20 ...,NaN,GRAMO,7.960967,MATERIAL HEMOSTATICO,18754341,CONTRATO ABIERTO NÚMERO 02-2022 ADQUISICIÓN DE...,Material medico
2,3,"SONDA LEVIN, No. 8 FR, DE PVC GRADO MÉDICO","SONDA LEVIN NASOGÁSTRICA, No. 8 FR, DE 40 A 42...",NaN,UNIDAD,3.055372,TUBOS Y DRENAJES: SONDAS DE TRACTO GASTROINTES...,18754341,CONTRATO ABIERTO NÚMERO 02-2022 ADQUISICIÓN DE...,Material medico
3,4,"SONDA LEVIN, No. 12 FR, DE PVC GRADO MÉDICO","SONDA LEVIN NASOGÁSTRICA, No.12 FR, DE 48 A 50...",NaN,UNIDAD,6.541582,TUBOS Y DRENAJES: SONDAS DE TRACTO GASTROINTES...,18754341,CONTRATO ABIERTO NÚMERO 02-2022 ADQUISICIÓN DE...,Material medico
4,5,"SONDA LEVIN, No. 10 FR, DE PVC GRADO MÉDICO","SONDA LEVIN NASOGÁSTRICA, No. 10 FR, DE 40 A ...",NaN,UNIDAD,3.900875,TUBOS Y DRENAJES: SONDAS DE TRACTO GASTROINTES...,18754341,CONTRATO ABIERTO NÚMERO 02-2022 ADQUISICIÓN DE...,Material medico


In [45]:
# Crear un nuevo data frame sin la columna caraceristicas y presentacion, prueba 1

BD = archivo.drop(columns=["Caracteristicas Solicitadas", "Presentacion","Nog"])
BD.head(5)

,Renglon,Nombre del Producto,Unidad de Medida,Precio de Referencia,Rubro,Evento,Categoria
0,1,CERA PARA HUESO (PALMITATO ISOPROPÍLICO),GRAMO,21.500000,MATERIAL HEMOSTATICO,CONTRATO ABIERTO NÚMERO 02-2022 ADQUISICIÓN DE...,Material medico
1,2,CERA PARA HUESO (JALEA DE PETROLATO),GRAMO,7.960967,MATERIAL HEMOSTATICO,CONTRATO ABIERTO NÚMERO 02-2022 ADQUISICIÓN DE...,Material medico
2,3,"SONDA LEVIN, No. 8 FR, DE PVC GRADO MÉDICO",UNIDAD,3.055372,TUBOS Y DRENAJES: SONDAS DE TRACTO GASTROINTES...,CONTRATO ABIERTO NÚMERO 02-2022 ADQUISICIÓN DE...,Material medico
3,4,"SONDA LEVIN, No. 12 FR, DE PVC GRADO MÉDICO",UNIDAD,6.541582,TUBOS Y DRENAJES: SONDAS DE TRACTO GASTROINTES...,CONTRATO ABIERTO NÚMERO 02-2022 ADQUISICIÓN DE...,Material medico
4,5,"SONDA LEVIN, No. 10 FR, DE PVC GRADO MÉDICO",UNIDAD,3.900875,TUBOS Y DRENAJES: SONDAS DE TRACTO GASTROINTES...,CONTRATO ABIERTO NÚMERO 02-2022 ADQUISICIÓN DE...,Material medico


In [13]:
#!python -m spacy download es_core_news_lg  # Descargar el modelo en español

In [46]:
nlp = spacy.load("es_core_news_lg")

In [51]:
# Crear una lista de tuplas con nombres de productos y etiquetas NER
productos_ner = [(nombre,categoria) for nombre, categoria in zip(BD['Nombre del Producto'], BD['Categoria'])]

# Inicializar el PhraseMatcher
matcher = PhraseMatcher(nlp.vocab)
patterns = [nlp(producto) for producto, _ in productos_ner]
matcher.add("PRODUCTOS", None, *patterns)

In [58]:
import json


# Convertir la lista de tuplas en el formato JSONL
jsonl_data = []
for nombre_producto, etiqueta_ner in productos_ner:
    jsonl_data.append({
        "text": nombre_producto,
        "spans": [{"start": 0, "end": len(nombre_producto), "label": etiqueta_ner}]
    })

# Escribir los datos en un archivo JSONL
with open("datos_etiquetados.jsonl", "w") as jsonl_file:
    for item in jsonl_data:
        jsonl_file.write(json.dumps(item) + "\n")

In [60]:
# Codigo para leer el archivo json creado
#jsonl_file = "datos_etiquetados.jsonl"

# Leer y procesar el archivo JSONL
#with open(jsonl_file, "r") as file:
#    for line in file:
#        data = json.loads(line)
#        print(data)

In [62]:
# Instalar Prodigy
!pip install prodigy

ERROR: Could not find a version that satisfies the requirement prodigy (from versions: none)
ERROR: No matching distribution found for prodigy


In [ ]:
import spacy
from spacy.matcher import PhraseMatcher

# Cargar el modelo de spaCy y crear el PhraseMatcher
nlp = spacy.load("es_core_news_sm")
matcher = PhraseMatcher(nlp.vocab)

# Lista de nombres de productos
nombres_productos = ["CERA PARA HUESO (PALMITATO ISOPROPÍLICO)", "SONDA LEVIN, No. 8 FR, DE PVC GRADO MÉDICO", ...]

# Convertir los nombres de productos en patrones para el PhraseMatcher
patrones = [nlp(nombre) for nombre in nombres_productos]

# Agregar los patrones al PhraseMatcher
matcher.add("PRODUCTOS", None, *patrones)

# Texto de ejemplo
texto = "La CERA PARA HUESO (PALMITATO ISOPROPÍLICO) es un ejemplo de material médico."

# Procesar el texto con spaCy
doc = nlp(texto)

# Encontrar las coincidencias en el texto
matches = matcher(doc)

# Crear entidades NER personalizadas a partir de las coincidencias
for match_id, start, end in matches:
    span = doc[start:end]
    label = "Material medico"
    span.merge(ent_type=label)

# Obtener las entidades NER personalizadas
entidades_ner = [(ent.text, ent.label_) for ent in doc.ents]

# Resultados
print(entidades_ner)
